
# Simulación de Clasificación de Cajas con Agentes en Jupyter Notebook

Este Jupyter Notebook implementa una simulación en la que agentes autónomos colaboran para clasificar cajas en pilas de un máximo de 5 cajas cada una. La simulación recopila información sobre el tiempo necesario hasta que todas las cajas están en su lugar y el número total de movimientos realizados por los robots.

![caricatura](robotandbox.jpg)


In [230]:

import agentpy as ap
import pathfinding as pf        #In case you want to use pathfinding algorithms for the agent's plan
import matplotlib.pyplot as plt
from owlready2 import *
import itertools
import random
import IPython
import math

# Ontologia de la simulación
Usando la libreria de Owlready2

In [264]:



onto.destroy(update_relation = True, update_is_a = True)

#Ontology name and path
onto = get_ontology("file:///content/robot_onto.owl")

#opening the ontology
with onto:

    #My SuperClass
    class Entity(Thing):
        pass

    class Robot(Entity):
        pass

    class Box(Entity):
        pass
    
    class Spot(Entity):
        pass

    class Place(Thing):
        pass

    #Property to describe the place of an entity in the grid
    class is_in_place(ObjectProperty):
        domain = [Entity]
        range = [Place]
        pass

    #Property that specifies the position of a Place
    class at_position(DataProperty,FunctionalProperty):
        domain = [Place]
        range = [str]
        pass

    #Property to describe how many boxes the agent can see
    class boxes_within_reach(ObjectProperty):
        domain = [Robot]
        range = [int]
        
    #Property to describe how many spots the agent can see
    class spots_within_reach(ObjectProperty):
        domain = [Robot]
        range = [int]


# Clase del agente Robot

In [265]:

class RobotAgent(ap.Agent):

    #BDI related functionality:

    #Perception
    def see(self,e):

        """ It gets all boxes (agentType 1) that the agent can see """

        #The range of observetion from the Robot's viewpoint,
        seeRange = self.model.p.worldSize[0]
        # seeRange = 20
        
        # bo = [a for a in e.neighbors(self, distance=seeRange) if a.agentType==1]
        # sp = [b for b in e.neighbors(self, distance=seeRange) if a.agentType==6]

        # #tupla de listas de boxes y spots
        # p = [bo,sp]
        
        p = [a for a in e.neighbors(self, distance=seeRange) if a.agentType==1]

        
        #lista de spots
        # ps = [a for a in e.neighbors(self, distance=seeRange) if a.agentType==6]

        return p


    #Belief revision function:
    def brf(self,p):

        """
        It will update the Beilef system of the agent.
        The beleif system is based on the Ontology
        """

        #Destroys previous beliefs
        for box in self.this_robot.boxes_within_reach:
            destroy_entity(box.is_in_place[0])
            destroy_entity(box)
        destroy_entity(self.this_robot.is_in_place[0])

        #Ontologically instantiate the robot
        currentPos = self.model.boxWorld.positions[self]
        self.this_robot.is_in_place = [Place(at_position = str(currentPos))]

        #Ontologically instantiate the boxes at reach
        # for c in p[0]:
        for c in p:
            theBox = Box(is_in_place = [Place()])
            theBox.is_in_place[0].at_position = str(self.model.boxWorld.positions[c])
            self.this_robot.boxes_within_reach.append(theBox)
            
         #Ontologically instantiate the spots 
        # for c in p[1]:
        #     theSpot = Spot(is_in_place = [Place()])
        #     theSpot.is_in_place[0].at_position = str(self.model.boxWorld.positions[c])
        #     self.this_robot.spots_within_reach.append(theSpot)


    #The options function (where it gets its Desires)
    def options(self):

        """ It returns the available goals to persue.
        These are based on each Box distance relative to the Robot
        """
        distances = {}

        #For each Box at reach from the Robot
        for onto_box in self.this_robot.boxes_within_reach:
            box_pos = eval(onto_box.is_in_place[0].at_position)
            robot_pos = eval(self.this_robot.is_in_place[0].at_position)
            #Calculate the euclidean distance:
            d = math.sqrt((box_pos[0]-robot_pos[0])**2 + (box_pos[1]-robot_pos[1])**2)
            #Store in a dictionary:
            distances[onto_box] = d

        return distances


    #The filter function (where it gets the Intention)
    def filter(self):

        """
        This will return the closest box as the target (Intention).
        It is based on the Robot's Desires.
        """

        #Sort the dictionary based on each box's distance:
        desires = {x:y for x,y in sorted(self.D.items(),key=lambda item:item[1])}

        #return the first box
        if desires:
            return list(desires.items())[0][0]
        else:
            return None


    #The plan function (Where the agent creates a plan)
    def plan(self):

        """
        Here the Robot will create a plan towards the current Intention.
        This returns a plan in the form of a list of tuples (x,y).
        Each step on the plan is a step over the grid on the main four
        directions (not diagonal).
        Example:
            [(1,0),(0,-1),(-1,0),(1,0),etc.]
        """

        if self.I == None:
            if random.randint(0,1) == 0:
                return [(random.choice([-1,1]),0)]
            elif random.randint(0,1) == 1:
                return [(0,random.choice([-1,1]))]
            else:
                return [(0,0)]

        thePlanX = []
        thePlanY = []

        #get target porition
        boxPos = eval(self.I.is_in_place[0].at_position)
        #get robot position
        robotPos = eval(self.this_robot.is_in_place[0].at_position)
        #calculate distances among each axis
        distance2D = (boxPos[0]-robotPos[0],boxPos[1]-robotPos[1])

        #create a list of atomic steps (1 or -1) on the X axis
        for i in range(abs(distance2D[0])):
            val = 1 if distance2D[0] >= 0 else -1
            thePlanX.append(val)

        #create a list of atomic steps (1 or -1) on the Y axis
        for j in range(abs(distance2D[1])):
            val = 1 if distance2D[1] >= 0 else -1
            thePlanY.append(val)

        #creates a list of tuples filling with ceros
        thePlanX = list(zip(thePlanX,[0 for _ in range(len(thePlanX))]))
        thePlanY = list(zip([0 for _ in range(len(thePlanY))],thePlanY))

        #Creates a final list of the whole plan and shuffles it
        #(The shuffling part is to have a less boring agent)
        thePlan = thePlanX + thePlanY
        random.shuffle(thePlan)

        return thePlan


    #The main BDI algorithm
    def BDI(self, p):

        """
        This function calls all functions from the BDI architecture.
        """

        #Calling brf at the beginning
        self.brf(p)

        #If the Robot reached a goal, then update Desires and Intentions,
        #and create new Plan
        if self.intentionSucceded:
            self.intentionSucceded = False
            self.D = self.options()
            self.I = self.filter()
            self.currentPlan = self.plan()


    #The function to execute actions
    def execute(self):

        """
        This function will execute the plan, action by action.
        Each action is a tuple that has a 1 or -1,
        so they describe if the agent needs to move in one
        direction or the other.
        """

        #If the plan hasn't finished
        if len(self.currentPlan) > 0:
            #Then  get the next action
            currentAction = self.currentPlan.pop()
        else: # If the plan has finished
            #It means the Robot has succeded in it current task
            self.intentionSucceded = True
            #So, then do nothing, until next new plan
            currentAction = (0,0)

        #Execute the selected action, using move_by()
        self.model.boxWorld.move_by(self,currentAction)


    #Initial beleifs function
    def initBeliefs(self,initPos):

        """
        This function will fill the Belief system, instantiating the first
        concepts form the ontology.
        """

        #initial Place instance
        place = Place(at_position=str(initPos))

        #initial Robot inistace
        self.this_robot = Robot(is_in_place = [place])


    #Initial intentions funtion
    def initIntentions(self):

        """
        This function will provide the first Intention,
        which in this case is empty.
        """

        self.intentionSucceded = True
        self.I = None


    #======================Main Agent's Functions======================================================================================================================


    #Setup
    def setup(self):

        #HIdentifier of the Robot Agent
        self.agentType = 0
        self.firstStep = True
        self.currentPlan = []


    #Step
    def step(self):

        #If it is the first step, then late-initialize
        if self.firstStep:
            initPos = self.model.boxWorld.positions[self]
            self.initBeliefs(initPos)
            self.initIntentions()
            self.firstStep = False

        #Execute the main BDI algorithm
        self.BDI(self.see(self.model.boxWorld))

        #Execute next action
        self.execute()

    #Update
    def update(self):
        pass

    #End
    def end(self):
        pass
    


# Clase del Agente Spot

In [266]:
#RECOLECCION AGENT

#No hace nada por que no se mueve
class SpotAgent(ap.Agent):

    #Setup
    def setup(self):
        #Identifier for Box Agent
        self.agentType = 6


    #Step
    def step(self):
        pass

    #Update
    def update(self):
        pass

    #End
    def end(self):
        pass
    

# Clase del agente Box

In [267]:
#BOX AGENT

#No hace nada por que no se mueve
class BoxAgent(ap.Agent):

    #Setup
    def setup(self):
        #Identifier for Box Agent
        self.agentType = 1


    #Step
    def step(self):
        pass

    #Update
    def update(self):
        pass

    #End
    def end(self):
        pass
    


# Modelo del mundo

In [268]:
#THE SIMULATION

class BoxWorldModel(ap.Model):

    #A function to get the amount of boxes left
    def get_boxes(self):
        return len(self.boxes)

    #Setup
    def setup(self):

        #Create robot agents
        self.robots = ap.AgentList(self,self.p.robotAgents,RobotAgent)

        #Create box agents
        self.boxes = ap.AgentList(self,self.p.boxAgents,BoxAgent)
        
        #Create spots agents
        self.spots = ap.AgentList(self,self.p.spotsAgents,SpotAgent)

        #Create the grid world
        self.boxWorld = ap.Grid(self,self.p.worldSize,track_empty=True)

        #Add agents to the grid
        self.boxWorld.add_agents(self.robots,random=True,empty=True)
        self.boxWorld.add_agents(self.boxes,random=True,empty=True)
        self.boxWorld.add_agents(self.spots,random=True,empty=True)


    #Step
    def step(self):

        #Do each agents step function
        self.robots.step()
        self.boxes.step()
        self.spots.step()


        #Verify if there is a collition between a Robot and a Box
        for robot in self.robots:
            for box in self.boxes:
                if box in self.boxWorld.positions and self.boxWorld.positions[robot] == self.boxWorld.positions[box]:
                    #If there is a collition then remove the Box from the grid, and from the simulation
                    self.boxWorld.remove_agents(box)
                    self.boxes.remove(box)
                    break

        #If there are no boxes left, then end the simulation
        if len(self.boxes) <= 0:
            self.stop()

    #Update
    def update(self):
        pass

    #End
    def end(self):
        pass
    


In [269]:
#funcion para animar la simulacion
def animation_plot(model, ax):
    agent_type_grid = model.boxWorld.attr_grid('agentType')
    
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Simulacion del modelo \n Tiempo necesario: {model.t}, "
                 f"Cajas faltantes: {model.get_boxes()}")
    


In [270]:
#parametros
#a random variables (0,1)
r = random.random()

#parameters dict
parameters = {
    "robotAgents" : 5,     #Cantidad de robots
    "boxAgents" : 50,      #Cantidad de cajas
    "spotsAgents" : 3,  #Cantidad de lugares de recoleccion
    "worldSize" : (30,30),      #Grid size
    "steps" : 1000,          #Max steps
    "seed" : 20*r           #seed for random variables 
}


In [271]:
print(model.t)

0


# Simulación

Se utiliza **matplotlib** y **agentpy** para realizar una simulación interactiva. Y se puede observar la animación final en formato HTML de los robots recogiendo las cajas y llevandolas a los puntos a donde se deben apilar. 


In [272]:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = BoxWorldModel(parameters)

#Run with animation
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
show = IPython.display.HTML(animation.to_jshtml())

show

# Resultados
Se muestra los siguientes resultado de la simulación en base a los parametros establecidos previamente.

1. Tiempo necesario:Tiempo requerido de los robots para recoger la totalidad de las cajas y regresarlos a los lugares de recolección.

2. Número de movimientos: Número de movimientos realizados por todos los robots para terminar la simulación.

In [273]:

print("Numero de movimientos :", model.run().info['completed_steps'])

Completed: 228 steps
Run time: 0:00:01.583483
Simulation finished
Numero de movimientos : 228
